In [1]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import csv
import seaborn as sns
from collections import Counter

import statistics as stats

from numpy.random import seed

import seaborn as sns; sns.set()

import os

import warnings

import pickle

from keras.models import load_model

from numpy import loadtxt

In [2]:
#Store CSV into Dataframe

df = "./Resources/train.csv"
train_df = pd.read_csv(df,encoding='latin-1')
train_df.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2.0,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0.0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3.0,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2.0,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1.0,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [3]:
#Check to see how many values are in the data frame
train_df.shape

(614, 13)

In [4]:
#Confirm null values
train_df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
# Confirm data types
train_df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents           float64
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [6]:
#Remove null values 
train_df = train_df.dropna(how='any',axis=0)

In [7]:
#Look for duplicated Loan IDs in the data set
train_df[train_df.duplicated(['Loan_ID'], keep=False)]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status


In [8]:
train_df.to_csv(r'/Users/aliciasmith/Desktop/Final_Project/Resources/train_df3.csv', index = False)

Machine Learning 

In [9]:
# Review dataframe info
# Can't have any Null values for ML
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            480 non-null    object 
 1   Gender             480 non-null    object 
 2   Married            480 non-null    object 
 3   Dependents         480 non-null    float64
 4   Education          480 non-null    object 
 5   Self_Employed      480 non-null    object 
 6   ApplicantIncome    480 non-null    int64  
 7   CoapplicantIncome  480 non-null    float64
 8   LoanAmount         480 non-null    float64
 9   Loan_Amount_Term   480 non-null    float64
 10  Credit_History     480 non-null    float64
 11  Property_Area      480 non-null    object 
 12  Loan_Status        480 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 52.5+ KB


In [10]:
# Turn all the categorical variables into numbers
from sklearn.preprocessing import LabelEncoder
category= ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status'] 
encoder= LabelEncoder()
for i in category:   
    train_df[i] = encoder.fit_transform(train_df[i]) 
    train_df.dtypes


In [11]:
# Review dataframe info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            480 non-null    object 
 1   Gender             480 non-null    int64  
 2   Married            480 non-null    int64  
 3   Dependents         480 non-null    int64  
 4   Education          480 non-null    int64  
 5   Self_Employed      480 non-null    int64  
 6   ApplicantIncome    480 non-null    int64  
 7   CoapplicantIncome  480 non-null    float64
 8   LoanAmount         480 non-null    float64
 9   Loan_Amount_Term   480 non-null    float64
 10  Credit_History     480 non-null    float64
 11  Property_Area      480 non-null    int64  
 12  Loan_Status        480 non-null    int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 52.5+ KB


In [12]:
# #lets convert the remaining metrics that are strings to numeric values so we can run our models
# Gender = {'Male':0, 'Female':1}
# train_df.Gender=[Gender[item] for item in train_df.Gender]

# Married = {'No':0, 'Yes':1}
# train_df.Married=[Married[item] for item in train_df.Married]

# Education = {'Not Graduate':0, 'Graduate':1}
# train_df.Education=[Education[item] for item in train_df.Education]

# Self_Employed = {'No':0, 'Yes':1}
# train_df.Self_Employed=[Self_Employed[item] for item in train_df.Self_Employed]

# Property_Area = {'Rural':0, 'Semiurban':1, 'Urban':2 }
# train_df.Property_Area=[Property_Area[item] for item in train_df.Property_Area]

# Loan_Status = {'N':0, 'Y':1}
# train_df.Loan_Status=[Loan_Status[item] for item in train_df.Loan_Status]



In [13]:
# Statistical summary of df. Used to give indication of data. 
train_df.describe()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,0.820833,0.647917,0.777083,0.202083,0.137500,5364.231250,1581.093583,144.735417,342.050000,0.854167,1.022917,0.691667
std,0.383892,0.478118,1.020815,0.401973,0.344734,5668.251251,2617.692267,80.508164,65.212401,0.353307,0.776411,0.462287
min,0.000000,0.000000,0.000000,0.000000,0.000000,150.000000,0.000000,9.000000,36.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,2898.750000,0.000000,100.000000,360.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,3859.000000,1084.500000,128.000000,360.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,2.000000,0.000000,0.000000,5852.500000,2253.250000,170.000000,360.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,3.000000,1.000000,1.000000,81000.000000,33837.000000,600.000000,480.000000,1.000000,2.000000,1.000000


In [14]:
# Percent of loans that were approved 
loans_approved = train_df[train_df['Loan_Status'] ==1]
len(loans_approved)
loans_approved_percent = len(loans_approved)/len(train_df) *100
loans_approved_percent

69.16666666666667

In [15]:
train_df.drop('Loan_ID', axis=1, inplace=True)

Neural Network Machine Learning Model 

In [16]:
#Prepare data before applying model 
#List column names
train_df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [17]:
#Target Variable = Loan_Status
X = train_df.drop("Loan_Status", axis=1)
y = train_df["Loan_Status"]
print(X.shape, y.shape)

(480, 11) (480,)


In [18]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
print(X.shape, y.shape)

(480, 11) (480, 2)


In [19]:
#scale the data before training the modle
# This is important so we normalize the data
# Ex: # of dependents ranges from 0-3 whereas Applicant income ranges from 0 to 81k? If we don't do this applicant income
# will dominate the other lower features which we don't want 

from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)

In [20]:
#splitting the data into testing and training sets 
from sklearn.model_selection import train_test_split

# X_train, X_test,y_train, y_test = train_test_split(X, y, random_state=1)
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=.5)
# X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=.1)
# X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=.25)
# X_train, X_test,y_train, y_test = train_test_split(X, y, random_state=1)


#random_state or test_size?

In [21]:
#print shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((240, 11), (240, 11), (240, 2), (240, 2))

In [22]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [23]:
#Create a sequential model
from tensorflow.keras.models import Sequential
nn_model = Sequential()

In [24]:

from tensorflow.keras.layers import Dense

nn_model.add(Dense(units=33, activation='relu', input_dim=11))
nn_model.add(Dense(units=66, activation='relu'))
# nn_model.add(Dense(units=132, activation='relu'))
# # nn_model.add(Dense(units=66, activation='relu'))
nn_model.add(Dense(units=2, activation='softmax'))

In [25]:
#summary of model 
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 33)                396       
_________________________________________________________________
dense_1 (Dense)              (None, 66)                2244      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 134       
Total params: 2,774
Trainable params: 2,774
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
# Fit (train) the model
nn_model.fit(
    X_train,
    y_train,
    epochs=500,
    verbose=2
)

Epoch 1/500
8/8 - 0s - loss: 0.7348 - accuracy: 0.3708
Epoch 2/500
8/8 - 0s - loss: 0.6372 - accuracy: 0.7458
Epoch 3/500
8/8 - 0s - loss: 0.5791 - accuracy: 0.7917
Epoch 4/500
8/8 - 0s - loss: 0.5413 - accuracy: 0.8125
Epoch 5/500
8/8 - 0s - loss: 0.5076 - accuracy: 0.8250
Epoch 6/500
8/8 - 0s - loss: 0.4800 - accuracy: 0.8375
Epoch 7/500
8/8 - 0s - loss: 0.4560 - accuracy: 0.8333
Epoch 8/500
8/8 - 0s - loss: 0.4394 - accuracy: 0.8292
Epoch 9/500
8/8 - 0s - loss: 0.4199 - accuracy: 0.8250
Epoch 10/500
8/8 - 0s - loss: 0.4063 - accuracy: 0.8292
Epoch 11/500
8/8 - 0s - loss: 0.3969 - accuracy: 0.8292
Epoch 12/500
8/8 - 0s - loss: 0.3883 - accuracy: 0.8333
Epoch 13/500
8/8 - 0s - loss: 0.3797 - accuracy: 0.8333
Epoch 14/500
8/8 - 0s - loss: 0.3735 - accuracy: 0.8375
Epoch 15/500
8/8 - 0s - loss: 0.3677 - accuracy: 0.8417
Epoch 16/500
8/8 - 0s - loss: 0.3631 - accuracy: 0.8417
Epoch 17/500
8/8 - 0s - loss: 0.3575 - accuracy: 0.8417
Epoch 18/500
8/8 - 0s - loss: 0.3539 - accuracy: 0.8417
E

Epoch 147/500
8/8 - 0s - loss: 0.0667 - accuracy: 0.9875
Epoch 148/500
8/8 - 0s - loss: 0.0615 - accuracy: 0.9875
Epoch 149/500
8/8 - 0s - loss: 0.0619 - accuracy: 0.9792
Epoch 150/500
8/8 - 0s - loss: 0.0601 - accuracy: 0.9792
Epoch 151/500
8/8 - 0s - loss: 0.0600 - accuracy: 0.9833
Epoch 152/500
8/8 - 0s - loss: 0.0582 - accuracy: 0.9833
Epoch 153/500
8/8 - 0s - loss: 0.0583 - accuracy: 0.9833
Epoch 154/500
8/8 - 0s - loss: 0.0574 - accuracy: 0.9792
Epoch 155/500
8/8 - 0s - loss: 0.0587 - accuracy: 0.9875
Epoch 156/500
8/8 - 0s - loss: 0.0587 - accuracy: 0.9875
Epoch 157/500
8/8 - 0s - loss: 0.0551 - accuracy: 0.9875
Epoch 158/500
8/8 - 0s - loss: 0.0551 - accuracy: 0.9875
Epoch 159/500
8/8 - 0s - loss: 0.0540 - accuracy: 0.9917
Epoch 160/500
8/8 - 0s - loss: 0.0534 - accuracy: 0.9833
Epoch 161/500
8/8 - 0s - loss: 0.0526 - accuracy: 0.9875
Epoch 162/500
8/8 - 0s - loss: 0.0530 - accuracy: 0.9875
Epoch 163/500
8/8 - 0s - loss: 0.0526 - accuracy: 0.9875
Epoch 164/500
8/8 - 0s - loss: 

Epoch 291/500
8/8 - 0s - loss: 0.0150 - accuracy: 1.0000
Epoch 292/500
8/8 - 0s - loss: 0.0145 - accuracy: 1.0000
Epoch 293/500
8/8 - 0s - loss: 0.0151 - accuracy: 1.0000
Epoch 294/500
8/8 - 0s - loss: 0.0148 - accuracy: 1.0000
Epoch 295/500
8/8 - 0s - loss: 0.0146 - accuracy: 1.0000
Epoch 296/500
8/8 - 0s - loss: 0.0141 - accuracy: 1.0000
Epoch 297/500
8/8 - 0s - loss: 0.0141 - accuracy: 1.0000
Epoch 298/500
8/8 - 0s - loss: 0.0140 - accuracy: 1.0000
Epoch 299/500
8/8 - 0s - loss: 0.0138 - accuracy: 1.0000
Epoch 300/500
8/8 - 0s - loss: 0.0143 - accuracy: 1.0000
Epoch 301/500
8/8 - 0s - loss: 0.0138 - accuracy: 1.0000
Epoch 302/500
8/8 - 0s - loss: 0.0136 - accuracy: 1.0000
Epoch 303/500
8/8 - 0s - loss: 0.0139 - accuracy: 1.0000
Epoch 304/500
8/8 - 0s - loss: 0.0137 - accuracy: 1.0000
Epoch 305/500
8/8 - 0s - loss: 0.0133 - accuracy: 1.0000
Epoch 306/500
8/8 - 0s - loss: 0.0142 - accuracy: 0.9958
Epoch 307/500
8/8 - 0s - loss: 0.0119 - accuracy: 1.0000
Epoch 308/500
8/8 - 0s - loss: 

Epoch 435/500
8/8 - 0s - loss: 0.0053 - accuracy: 1.0000
Epoch 436/500
8/8 - 0s - loss: 0.0053 - accuracy: 1.0000
Epoch 437/500
8/8 - 0s - loss: 0.0051 - accuracy: 1.0000
Epoch 438/500
8/8 - 0s - loss: 0.0052 - accuracy: 1.0000
Epoch 439/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 440/500
8/8 - 0s - loss: 0.0052 - accuracy: 1.0000
Epoch 441/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 442/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 443/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 444/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 445/500
8/8 - 0s - loss: 0.0050 - accuracy: 1.0000
Epoch 446/500
8/8 - 0s - loss: 0.0049 - accuracy: 1.0000
Epoch 447/500
8/8 - 0s - loss: 0.0048 - accuracy: 1.0000
Epoch 448/500
8/8 - 0s - loss: 0.0051 - accuracy: 1.0000
Epoch 449/500
8/8 - 0s - loss: 0.0052 - accuracy: 1.0000
Epoch 450/500
8/8 - 0s - loss: 0.0052 - accuracy: 1.0000
Epoch 451/500
8/8 - 0s - loss: 0.0045 - accuracy: 1.0000
Epoch 452/500
8/8 - 0s - loss: 

**Quantifying the DNN Model**

In [28]:
model_loss, model_accuracy = nn_model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 3.8063 - accuracy: 0.6542
Normal Neural Network - Loss: 3.806300163269043, Accuracy: 0.6541666388511658


In [29]:
model_loss, model_accuracy = nn_model.evaluate(
    X_train, y_train, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.0033 - accuracy: 1.0000
Normal Neural Network - Loss: 0.003279800759628415, Accuracy: 1.0


In [30]:
# import joblib
# filename = 'finalnn_model.sav'
# joblib.dump(model_accuracy, filename)

In [31]:
# # save model and architecture to single file
# nn_model.save("nn_model.h5")

In [32]:
# save the model to disk
filename = 'nn_model_model.sav'
pickle.dump(nn_model, open(filename, 'wb'))

TypeError: cannot pickle 'weakref' object

In [ ]:
# print("Saved model to disk")

In [ ]:
# # load model
# nn_model = load_model('nn_model.h5')
# # summarize model.
# nn_model.summary()

In [ ]:
# model_loss, model_accuracy = nn_model.evaluate(
#     X_test, y_test, verbose=2)
# print(
#     f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# model_loss, model_accuracy = nn_model.evaluate(
#     X_train, y_train, verbose=2)
# print(
#     f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")